## Notebook for sample QC of WGS sample cohort

### actually sample QC of WGS has already been run for this

just apply previous sample QC from /labshare/raph/notebooks/wgs/pd/nabec_wgs.ipynb 

#### library imports and global variables

In [4]:
# import libraries
import threading
import pandas as pd
import os
pd.set_option('display.max_columns', None)

In [8]:
# global notebook variables for both python and bash magic (by stdin arguments)
in_dir = '/labshare/raph/datasets/amppd'
wrk_dir = '/labshare/anni/ipdgc/KTN1'
cohort = 'amppd'
build = 'nov2020'
cohort_build = f'{cohort}.{build}'

autosomes = [str(x) for x in list(range(1,23))]
sexomes = ['X']
chromosomes = autosomes + sexomes

qc_dir = f'{wrk_dir}/qc'
genotypes_dir = f'{wrk_dir}/genotypes'
qc_bfile_set = f'{qc_dir}/{cohort_build}.geno05maf05hwe000001'
eqtl_info_dir = f'{wrk_dir}/sample_info'
#sqc_excludes_file = f'/labshare/raph/datasets/rosmap/wgs/qc/{cohort}.nov2018.excludes.list'
sample_info_file = f'{in_dir}/sample_info/clinical/Demographics.csv'


min_age = 15

#### threading related functions

In [2]:
#### threading related function

# run command line procsss with bash majic
# make this a small function so it can be target function for threading
def run_bash_cmd(this_cmd):
    !{this_cmd}

# for each bash command is list add to execution thread, join and wait til all done
def run_bash_cmds_threaded(cmd_list):
    job_threads = []
    for this_cmd in cmd_list:
        this_thread = threading.Thread(target=run_bash_cmd, args=(this_cmd,))
        job_threads.append(this_thread)
        this_thread.start()

    for job_thread in job_threads:
        job_thread.join()          

#### subset out the analysis ready, Euro centric, pfiles for eQTL analysis

#### combine the sqc samples and too young subjects as plink remove file 

In [10]:
# load the sample info file and make sure excluding young subjects as well
info_df = pd.read_csv(sample_info_file)
print(info_df.shape)
info_df.head()

(4298, 9)


participant_id        GUID visit_name  visit_month  age_at_baseline     sex  \
0  PD-PDNE299YPT  PDNE299YPT         M0            0               74    Male   
1  PD-PDYW828VAV  PDYW828VAV         M0            0               57  Female   
2  PD-PDWV958JPG  PDWV958JPG         M0            0               68    Male   
3  PD-PDRF387ZV3  PDRF387ZV3         M0            0               68    Male   
4  PD-PDAC268KWV  PDAC268KWV         M0            0               69  Female   

                ethnicity   race  education_level_years  
0  Not Hispanic or Latino  White  Greater than 16 years  
1  Not Hispanic or Latino  White            12-16 years  
2  Not Hispanic or Latino  White  Greater than 16 years  
3  Not Hispanic or Latino  White            12-16 years  
4  Not Hispanic or Latino  White  Greater than 16 years

In [13]:
#info_df['age_at_baseline'] = info_df['age_at_baseline'].str.replace('+','')
#info_df['age_at_baseline']=info_df['age_at_baseline'].astype(float)
exclude_too_young = info_df.loc[info_df['age_at_baseline'] < min_age,['participant_id']]
print(exclude_too_young.shape)

# # read the sqc samples
# excludes_df = pd.read_csv(sqc_excludes_file, header=None)
# print(excludes_df.shape)

# get intersect union of sqc and too young
#exclude_ids = set(excludes_df[0]) | set(exclude_too_young['participant_id'])
exclude_ids =  set(exclude_too_young['participant_id'])
print(len(exclude_ids))

(0, 1)
0


In [ ]:
## convert all wgs to plink files
#for chrom in autosomes:
# for chrom in autosomes:
#     print(f'plink2 --vcf rosmap.chr{chrom}.vcf.gz --make-pgen --out rosmap.chr{chrom} &')

In [8]:
# def frmt_plink2_subset(genos_dir, in_name, out_name, chrom, excludes_file):
#     if in_name == out_name:
#         print('in name cannot be same of out name, here')
#         plink_cmd = '#error'
#     else:    
#         in_pfiles = f'{wrk_dir}/wgs/{in_name}.chr{chrom}'
#         out_pfiles = f'{wrk_dir}/wgs/{out_name}.chr{chrom}'
#         filter_pass = ' --var-filter'
#         plink_cmd = f'plink2 --pfile {in_pfiles} --remove {excludes_file} \
# --silent --make-pgen --out {out_pfiles}'
#     return plink_cmd

# def frmt_plink2_subset(genos_dir, in_name, out_name, chrom):
#     if in_name == out_name:
#         print('in name cannot be same of out name, here')
#         plink_cmd = '#error'
#     else:    
#         in_pfiles = f'{wrk_dir}/wgs/{in_name}.chr{chrom}'
#         out_pfiles = f'{wrk_dir}/wgs/{out_name}.chr{chrom}'
#         filter_pass = ' --var-filter'
#         plink_cmd = f'plink2 --pfile {in_pfiles} \
# --silent --make-pgen --out {out_pfiles}'
#     return plink_cmd

# # # save the plink removes files
# # excludes_df = pd.DataFrame(data=exclude_ids, columns=['#FID'])
# # excludes_df['IID'] = excludes_df['#FID']
# # remove_file = f'{eqtl_info_dir}/rosmap.sept2020.excludes.plink'
# # excludes_df.to_csv(remove_file, index=False, sep='\t')

# cmds = [frmt_plink2_subset(genotypes_dir, f'{cohort}', cohort_build, chrom) for chrom in chromosomes] 

# # print(cmds)
# # now run the cmds concurrently    
# run_bash_cmds_threaded(cmds)  

Error: Failed to open /labshare/anni/eqtl/rosmap/wgs/rosmap.chrX.pgen.


In [17]:

# def frmt_plink2_subset(genos_dir, in_name, out_name, chrom):
#     if in_name == out_name:
#         print('in name cannot be same of out name, here')
#         plink_cmd = '#error'
#     else:    
#         in_pfiles = f'{wrk_dir}/wgs/{in_name}.chr{chrom}'
#         out_pfiles = f'{wrk_dir}/wgs/{out_name}.chr{chrom}'
#         filter_pass = ' --var-filter'
#         plink_cmd = f'plink2 --pfile {in_pfiles} \
# --silent --make-bed --out {out_pfiles}'
#     return plink_cmd

# # # save the plink removes files
# # excludes_df = pd.DataFrame(data=exclude_ids, columns=['#FID'])
# # excludes_df['IID'] = excludes_df['#FID']
# # remove_file = f'{eqtl_info_dir}/rosmap.sept2020.excludes.plink'
# # excludes_df.to_csv(remove_file, index=False, sep='\t')

# cmds = [frmt_plink2_subset(genotypes_dir, f'{cohort}', cohort_build, chrom) for chrom in chromosomes] 

# # print(cmds)
# # now run the cmds concurrently    
# run_bash_cmds_threaded(cmds)  

Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr5.bim cannot contain multiallelic variants.
Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr6.bim cannot contain multiallelic variants.
Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr1.bim cannot contain multiallelic variants.
Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr9.bim cannot contain multiallelic variants.
Error: Failed to open /labshare/anni/eqtl/rosmap/wgs/rosmap.chrX.pgen.
Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr7.bim cannot contain multiallelic variants.
Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr22.bim cannot contain multiallelic variants.
Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr13.bim cannot contain multiallelic variants.
Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr8.bim cannot contain multiallelic variants.
Error: /labshare/anni/eqtl/rosmap/wgs/rosmap.oct2020.chr2.bim cannot contain multiallelic variants.
Error: /labshare/anni/eqtl/

#### generate the with cohort genetic PCs covariates

In [9]:
cohort_build

'rosmap.oct2020'

In [17]:
# #cohort = 'ppmi'
# #cohort = 'pdbp'
# cohort = 'biofind'

# qc_dir = f'{wrk_dir}/qc'
# genotypes_dir = f'{wrk_dir}/genotypes'
# qc_bfile_set = f'{qc_dir}/{cohort}.geno05maf05hwe000001'

# def filter_chroms(indir, outdir, chrom):
#     plink_cmd = f'plink2 --pfile {indir}/{cohort}.freeze9.chr{chrom}\
#     --silent --geno 0.05 --maf 0.05 --hwe 0.000001 --make-pgen \
#     --out {outdir}/{cohort}.freeze9.chr{chrom}.geno05maf05hwe000001'
#     return plink_cmd

# cmds = [filter_chroms(f'{in_dir}/genotypes', genotypes_dir, chrom) for chrom in chromosomes] 

# #print(cmds)
# # now run the cmds concurrently    
# run_bash_cmds_threaded(cmds)  

--geno, and/or applying different p-value thresholds to distinct subsets of
your data.


In [21]:
# ## merge chrom files
# autosomes_sub = [str(x) for x in list(range(2,23))]
# #cohort = 'ppmi'
# cohort = 'pdbp'
# #cohort = 'biofind'

# #make plink lists
# all_strings = []
# for x in chromosomes:
#     string = f'{wrk_dir}/genotypes/{cohort}.freeze9.chr{x}.geno05maf05hwe000001'
#     all_strings.append(string)
    
# with open(f'{wrk_dir}/genotypes/{cohort}.freeze9.plink.files', 'w') as f:
#     for item in all_strings:
#         f.write("%s\n" % item)  
        
# plink_cmd = f'plink2 --pfile 

In [23]:
# cohorts = ['ppmi','pdbp','biofind']

# for x in cohorts:
#     cp_cmd = f'{in_dir}/qc/{x}.freeze9.geno05maf05hwe000001* {wrk_dir}/genotypes/'
#     !{cp_cmd}

/usr/bin/sh: /labshare/raph/datasets/amppd/qc/ppmi.freeze9.geno05maf05hwe000001.bed: Permission denied
/usr/bin/sh: /labshare/raph/datasets/amppd/qc/pdbp.freeze9.geno05maf05hwe000001.bed: Permission denied
/usr/bin/sh: /labshare/raph/datasets/amppd/qc/biofind.freeze9.geno05maf05hwe000001.bed: Permission denied


In [26]:
cohorts = ['ppmi','pdbp','biofind']

for x in cohorts:
    print(f'{wrk_dir}/genotypes/{x}.freeze9.geno05maf05hwe000001')

/labshare/anni/ipdgc/KTN1/genotypes/ppmi.freeze9.geno05maf05hwe000001
/labshare/anni/ipdgc/KTN1/genotypes/pdbp.freeze9.geno05maf05hwe000001
/labshare/anni/ipdgc/KTN1/genotypes/biofind.freeze9.geno05maf05hwe000001


In [30]:
##merge cohort files

##in /labshare/anni/ipdgc/KTN1/genotypes
merge_cmd = f'plink --bfile biofind.freeze9.geno05maf05hwe000001 --merge-list merge.list.file --make-bed --out biofind.pdbp.ppmi.freeze9.geno05maf05hwe000001'
print(merge_cmd)
print()
##convert to plink2 file
cov_cmd = f'plink2 --bfile biofind.pdbp.ppmi.freeze9.geno05maf05hwe000001 --make-pgen --out biofind.pdbp.ppmi.freeze9.geno05maf05hwe000001'
print(cov_cmd)

plink --bfile biofind.freeze9.geno05maf05hwe000001 --merge-list merge.list.file --make-bed --out biofind.pdbp.ppmi.freeze9.geno05maf05hwe000001

plink2 --bfile biofind.pdbp.ppmi.freeze9.geno05maf05hwe000001 --make-pgen --out biofind.pdbp.ppmi.freeze9.geno05maf05hwe000001


In [32]:
cohort = 'biofind.pdbp.ppmi'
qc_bfile_set = f'{wrk_dir}/genotypes/biofind.pdbp.ppmi.freeze9.geno05maf05hwe000001'


os.makedirs(eqtl_info_dir, exist_ok=True)
# find pruned variants
!plink2 --pfile {qc_bfile_set} --silent \
--indep-pairwise 50 5 0.3 --out {qc_dir}/{cohort}.freeze9.ldprune 

# subset to pruned variants
!plink2 --pfile {qc_bfile_set} --silent \
--extract {qc_dir}/{cohort}.freeze9.ldprune.prune.in \
--make-pgen --out {qc_dir}/{cohort}.freeze9.pruned

# generate pca's (plink approx from fastpca)
!plink2 --pfile {qc_dir}/{cohort}.freeze9.pruned \
--silent --pca 20 \
--out {qc_dir}/{cohort}.freeze9.pca


In [35]:
!head -n 5 {qc_dir}/{cohort}.freeze9.pca.eigenvec

#FID	IID	PC1	PC2	PC3	PC4	PC5	PC6	PC7	PC8	PC9	PC10	PC11	PC12	PC13	PC14	PC15	PC16	PC17	PC18	PC19	PC20
BF-1001	BF-1001	-0.0103983	-0.0125451	-0.0014231	-0.0107377	-0.002274	0.00025784	0.000189628	-0.00175893	0.0429792	0.0205714	-0.0274269	0.00582651	0.0063316	-0.0183071	0.00163519	0.0085687	0.0231381	-0.00262803	0.0100802	0.00964221
BF-1002	BF-1002	-0.00886708	-0.00951145	-0.00252233	-0.00920649	-0.00417999	-0.00229887	0.00571876	-0.00587657	-0.0132618	0.0340973	-0.0258789	0.0130329	0.00584489	0.00515474	0.00474676	0.00820733	0.00204972	0.00641643	-0.0059922	0.000149631
BF-1003	BF-1003	-0.00769985	-0.00856688	-0.000252177	0.000264775	-0.00225617	0.0063823	0.00201952	0.00667474	0.00782051	-0.0121499	0.00991363	-0.00504607	-0.00499437	-0.00964764	-0.00656234	0.00449766	0.00384023	-0.00108994	0.000621527	-0.000366019
BF-1004	BF-1004	-0.00867631	-0.00957124	-0.00182781	-0.00606657	-0.00351643	-0.00897248	-0.000363337	0.00433838	0.028271	0.00382232	0.0179637	0.00014846	0.00393683	0.00532348	-0